# LANL Earthquake prediction

2019/04/10

This is the very beginning of the challenge for LANL Earthquake prediction.
While downloading first look into data addtional information

![](./Capture1.JPG)

So data should be a mix of some periodic and aperiodic signals.

From the data description
### Data fields
acoustic_data - the seismic signal [int16]<br>
time_to_failure - the time (in seconds) until the next laboratory earthquake [float64]<br>
seg_id - the test segment ids for which predictions should be made (one prediction per segment)<br>
<br>
So it is one dimensional int16 input and continous output to predict.

##### Trying to find some useful kernels but I got this....

![Alt](./Capture2.JPG)

Rest in peace...

### Look into discussions

From host there is a introdution thread:
https://www.kaggle.com/c/LANL-Earthquake-Prediction/discussion/77525#latest-500808

They listed three papers of their research, which dealt with tha same data for this competition.
In their first paper: https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1002/2017GL074677

From their abstract:
>  These predictions are based __solely on the instantaneous physical characteristics of the acousticalsignal and do not make use of its history__. Surprisingly, machine learning identifies a signal emitted from thefault zone previously thought to be low-amplitude noise that enables failure forecasting throughout thelaboratory quake cycle. We infer that this signal originates from continuous grain motions of the fault gougeas the fault blocks displace.

Their method:
![](./Capture3.JPG)

So they use a 1000-trees Random Forrest to predict the experimental failure.